In [1]:
import pdfplumber

# path = "~/Documents/GitHub/Order-Assistant/StoreReports-1.pdf"
path = 'StoreReports-1.pdf'

# all files are formatted the same
PAGES = 4

In [ ]:
class items:
    identified = False
    begin = 0.00
    delivery = 0.00
    end = 0.00
    used = 0.00
    cost = 0.00
    net = 0.00

    def __init__(self, name) -> str:
        self.name = name

    def checked(self, id) -> bool:
        self.identified = id

    def addData(self, begin, delivery, end, used, cost, net):
        self.begin = begin
        self.delivery = delivery
        self.end = end
        self.used = used
        self.cost = cost
        self.net = net


In [ ]:
drinks = list(items)
drinks.append(items('591 Pepsi'))

In [2]:
# list of all items 
drinks = {
    '591ml Pepsi' : False,
    '591ml Diet Pepsi' : False,
    '591ml Mountain Dew' : False,
    '591ml Diet Mtn Dew' : False,
    '591ml Aquafina' : False,
    '591ml Dr Pepper' : False,
    '591ml Mug Root Beer' : False,
    '591ml Schw Gngr Ale' : False,
    '591ml WCherry Pepsi' : False,
    '591ml Crush Orange' : False,
    '591ml Seven Up' : False,
    '591ml Brsk IceT' : False,
    '2Ltr Pepsi' : False,
    '2Ltr Diet Pepsi' : False,
    '2Ltr Mountain Dew' : False,
    '2Ltr Dt Mtn Dew' : False,
    '2Ltr Dr Pepper' : False,
    '2Ltr Mug Root Beer' : False,
    '2Ltr Schw Gngr Ale' : False,
    '2Ltr Crush Orange' : False,
    '2Ltr Brsk SwTea Lm' : False,
    '2Ltr Seven Up' : False,
    '355ml Pepsi' : False,
    '355ml Diet Pepsi' : False,
    '355ml Diet Mtn Dew' : False,
    '355ml Dr Pepper' : False,
    '355ml Schw Gngr Ale' : False,
    '355ml Brsk SwTea Lm' : False,
    '355ml Seven Up' : False,
    '12oz Welch Gr Soda' : False
}

cheese = {
    'Cheddar Cheese' : False,
    'Three Cheese Blend' : False,
    'String Cheese' : False,
    'Agropur Cheese' : False,
    '2 Cheese' : False,
    '3 Cheese' : False,
    'Feta Cheese' : False,
    'Philly Whipped Cream Chee' : False

}

dough = {
    'Dough Tray 10': False,
    'Dough Tray 12': False,
    'Dough Tray 14': False,
    'Dough Tray 16': False,
    'Chocolate Chip Cookie': False,
    'Choc Chip Brownie': False,
    'Cinnamon Pull Apart': False,
    'Thin Crust 14': False,
    'Gluten Free Crust': False
}

# There are two pepperonis in the log, needs
meats = {
    'Pepperoni' : False,
    'Bacon' : False,
    'Salami' : False,
    'Italian Sausage' : False,
    'Anchovies' : False,
    'Grilled Chicken' : False,
    'Chicken Popper' : False,
    'Salami' : False,
    'Canadian Bacon' : False,
    'Philly Cheesesteak' : False,
    'Sliced Spicy Meatballs' : False,
    'Meatball' : False,
    'Beef' : False,
    'Sausage' : False,
    'Italian Sausage' : False,
    'Roasted Wings' : False,
    'Donair Topping' : False
}

produce = {
    'Banana Peppers' : False,
    'Black Olives' : False,
    'Pineapple Lg Can' : False,
    'Mushrooms' : False,
    'Fresh Spinach' : False,
    'Green Peppers' : False,
    'Onions' : False,
    'Tomatoes' : False,
    'Banana Peppers' : False,
    'Pepperoncini Peppers' : False,
    'Jalapeno Peppers' : False,
    'Green Olives' : False
}

sauces = {
    'Bulk Ranch Sauce': False,
    'Pizza Sauce': False,
    'Jug Garlic Sauce': False,
    'Garlic Sauce Cups': False,
    'Pizza Sauce Cups': False,
    'Cheese Sauce Cups': False,
    'Ranch Sauce Cups': False,
    'Barbecue Sauce Cups': False,
    'Buffalo Sauce Cups': False,
    'Honey Must Sauce Cups': False,
    'Blue Cheese Sauce Cups': False,
    'Barbecue Sauce': False,
    'Garlic Parmesan Sauce Jug': False,
    'Buffalo Sauce': False,
    'Pouch Honey Chptl': False,
    'BBQ Bulk': False,
    'Alfredo Sauce': False,
    'Spinach Alfredo Sauce': False,
    'Cream Cheese Flavored Ici': False,
    'Creamy Garlic Dipping Cup': False,
    'Donair Sauce': False,
    'Hellman\'s Bulk Ranch': False
}

seasoning = {
    'Italian Seasoning' : False,
    'Seasoning Packets' : False,
    'Parmesan' : False,
    'Crushed Red Pepper' : False,
    'Dustinator' : False
}

boxes = {
    'Pizza Box 10': False,
    'Pizza Box 12': False,
    'Pizza Box 14': False,
    'Pizza Box 16': False,
    'Pizza Slice Box': False
}

In [3]:
category_dict = {
    'BEVERAGES' : drinks,
    'CHEESE' : cheese,
    'MEATS' : meats,
    'PRODUCE' : produce,
    'DOUGH' : dough,
    'SAUCES' : sauces,
    'SEASONS/FLOUR' : seasoning,
    'BOXES' : boxes
}

In [ ]:
# there are 6 required float numbers form the data form
def getData():
    
    return False

In [4]:
# TODO pepperoni handling
def pepperoniHandling():
    details = [0.00, 0.00, 0.00, 0.00, 0.00, 0.00]
    return details


In [5]:
PAGES = 5
category = ''
value_flag = False
value_counter = 0

for page in range(PAGES):
    with pdfplumber.open(path) as pdf:
        page = pdf.pages[page]
        text = page.extract_text()

    print(f'Reading : {page}')
    word = ''
    space_counter = 0

    for character in text:
        if ord(character) != 160:                                                   # 160 is the hex value for ' '
            word += character
            space_counter = 0

        elif len(word) > 0: 
            word += ' '
            space_counter += 1

            if space_counter == 2:                                                  # word ends if there are more than one space between words
                word = word.strip()
                # print (word)
                
                if word in category_dict:                                           # a new category will only appear once all items have been read
                    for item in category:
                        if not category[item]:
                            print(f'** WARNING ** did not log {item}')

                    # print(f'category : {word}')
                    category = category_dict[word]

                elif word in category:
                    value_flag = True 
                    print('Value Flag True')
                    if word == 'Pepperoni':
                        pepperoniHandling()

                    category[word] = True

                elif value_flag:
                    print(word)       
                    value_counter += 1

                    
                word = ''    
                space_counter = 0       
print('File Complete')

Reading : <Page:1>
Reading : <Page:2>
Reading : <Page:3>
Reading : <Page:4>
Reading : <Page:5>
File Complete


In [6]:
print(text)

09/19/2021                                           PAPA JOHNS PIZZA ­ RESTAURANT 11532                                         Page 3 of 4
04:04:30 pm                                      Actual Inventory Cost Report ­ Detailed View                                       INVCOSRP
                                                           09/13/2021 ­ 09/19/2021
    Item # Description                        Begin  Deliv./Xfr        End       Used         Cost      %  of Net
    SAUCES
    1002   Bulk Ranch Sauce                    0.00        0.00       0.00       0.00         0.00           0.00
    1006   Pizza Sauce                         5.00       48.00       0.00      53.00       240.91           1.53
    1104   Jug Garlic Sauce                    3.10        2.00       0.00       5.10         3.75           0.02
    1105   Garlic Sauce Cups                   5.50        6.00       0.00      11.50       141.09           0.89
    1112   Pizza Sauce Cups                    0.00   